In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
import pickle
from sklearn.cluster import KMeans
warnings.filterwarnings('ignore')

In [2]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

def t2h(text):
    return transliterate(text,sanscript.TELUGU,sanscript.DEVANAGARI)

In [3]:
stopwords=['కూడా', 'ఉన్నారు', 'ఎవరైనా', 'ఎప్పుడు', 'ఎక్కడైనా', 'అందుబాటులో', 'ఒక ప్రక్కన', 'సంబంధం', 'మాత్రమే', 'అడగండి', 'గా', 'మధ్య', 'ప్రకారం', 'అనుమతించు', 'మెచ్చుకో', 'వద్ద', 'తగిన', 'అడ్డంగా', 'అడగడం', 'ఇప్పటికే', 'కనిపిస్తాయి', 'కాదు', 'మరియు', 'మరొక', 'అనుగుణంగా', 'అయితే', 'చేయగలిగింది', 'దాదాపు', 'గురించి', 'పై', 'వెంట', 'వేరుగా', 'చుట్టూ', 'దూరంగా', 'ఏ', 'నిజంగా', 'అనుమతిస్తుంది', 'అందరూ', 'ఎవరో ఒకరు', 'వ్యతిరేకంగా', 'ఏదైనా', 'తర్వాత', 'మళ్ళీ', 'ఏమైనప్పటికి', 'ఒక']

In [4]:
vowels=set("అ ఆ ఇ ఈ ఉ ఊ ఋ ౠ ఌ ౡ ఎ ఏ ఐ ఒ ఓ ఔ అం అః".split())

conso="క	ఖ	గ	ఘ	ఙ	చ	ఛ	జ	ఝ	ఞ	ట	ఠ	డ	ఢ	ణ	త	థ	ద	ధ	న	ప	ఫ	బ	భ	మ	య	ర	ల	వ	శ	ష	స	హ	ళ	క్ష	ఱ"

matra = "ఀ	ఁ	ం	ః	ఄ	ఽ	ా	ి	ీ	ు	ూ	ృ	ౄ	ె	ే	ై	ొ	ో	ౌ	్	ౕ	ౖ	ౢ	ౣ"

halantha='్'

In [5]:
consonants=set(conso.split("\t"))
matra = set(matra.split("\t"))

vowels = list(vowels)
consonants=list(consonants)
matra=list(matra)

alphabets = vowels+consonants

In [6]:
delimiters=["।","?","!","."]

In [7]:
file=open('glove.pkl','rb')
glove=pickle.load(file)
file.close()

In [8]:
file=open('sentence.pkl','rb')
verse_embeddings_sentence=pickle.load(file)
file.close()
file=open('max.pkl','rb')
verse_embeddings_max=pickle.load(file)
file.close()
file=open('mean.pkl','rb')
verse_embeddings_mean=pickle.load(file)
file.close()

In [9]:
file=open('whole.pkl','rb')
verse_embeddings_whole=pickle.load(file)
file.close()

In [10]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    regexPattern = '.'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [11]:
def word_embedding(word):
    try:
        return glove[t2h(word)]
    except:
        return np.array([0.0]*100)

In [12]:
def cosine_similarity(a,b):
    try:
        return a.dot(b)/(lin.norm(a)*lin.norm(b))
    except:
        return -1.0

In [13]:
def ed(a,b):
    return lin.norm(a-b)

In [14]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [15]:
data=pd.read_csv('../telugu.csv')

In [16]:
query='నాకు చాలా సమస్యలు ఉన్నాయి'
query_embedding=sentence_embedding(query)
query_embedding_list=query_embedding.reshape(1,-1).astype(np.float64)

# Max Strategy

## Cosine Similarity

In [17]:
def max_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [18]:
verse_index=max_cosine(query_embedding)

In [19]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...
399,10,35,बृहत्साम तथा साम्नां गायत्री छन्दसामहम्।\n\nमा...,"ఇంతకు పూర్వం శ్రీ కృష్ణుడు వేదములలో, అద్భుతమైన..."
652,18,37,यत्तदग्रे विषमिव परिणामेऽमृतोपमम्।तत्सुखं सात्...,భారతదేశ ఉసిరికాయ (ఆమ్లా) ఆరోగ్యానికి లాభకారియై...
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
646,18,31,यया धर्ममधर्मं च कार्यं चाकार्यमेव च।अयथावत्प्...,వ్యక్తిగత మమకారాసక్తుల వలన రాజసిక బుద్ధి మిశ్ర...
570,16,7,प्रवृत्तिं च निवृत्तिं च जना न विदुरासुराः।न श...,ధర్మము అంటే ఒక వ్యక్తి యొక్క పరిశుద్ధతకు మరియు...
403,10,39,यच्चापि सर्वभूतानां बीजं तदहमर्जुन।\n\nन तदस्त...,సమస్త సృష్టికి నిమిత్త కారణము (efficient cause...
595,17,8,आयुःसत्त्वबलारोग्यसुखप्रीतिविवर्धनाः।रस्याः स्...,"14వ అధ్యాయము, 6వ శ్లోకములో, శ్రీ కృష్ణుడు సత్త..."
589,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,"ఎవ్వరూ కూడా శ్రద్ధ/విశ్వాసము లేకుండా ఉండరు, ఎం..."


## Euclidean Distance

In [20]:
def max_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [21]:
verse_index=max_ed(query_embedding)

In [22]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
589,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,"ఎవ్వరూ కూడా శ్రద్ధ/విశ్వాసము లేకుండా ఉండరు, ఎం..."
570,16,7,प्रवृत्तिं च निवृत्तिं च जना न विदुरासुराः।न श...,ధర్మము అంటే ఒక వ్యక్తి యొక్క పరిశుద్ధతకు మరియు...
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,"లభతే అంటే ""వారు పొందెదరు"" అని. దేవతల యొక్క భ..."
262,6,37,अर्जुन उवाच\n\nअयतिः श्रद्धयोपेतो योगाच्चलितमा...,భగవత్ ప్రాప్తి దిశగా ప్రయాణం 'శ్రద్ధ' తోనే మొద...
459,11,53,नाहं वेदैर्न तपसा न दानेन न चेज्यया।\n\nशक्य ए...,"అర్జునుడికి విశ్వ రూపము చూపించిన పిదప, ఆ విశ్వ..."
458,11,52,श्री भगवानुवाच\n\nसुदुर्दर्शमिदं रूपं दृष्टवान...,"అర్జునుడికి విశ్వ రూపము చూపించిన పిదప, ఆ విశ్వ..."
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
656,18,41,ब्राह्मणक्षत्रियविशां शूद्राणां च परंतप।कर्माण...,సరిగ్గా సరిపోయే వృత్తి దొరకటం అంటే సరిగ్గా సరి...
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...


# Mean Pooling

## Cosine Similarity

In [23]:
def mean_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [24]:
verse_index=mean_cosine(query_embedding)

In [25]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...
399,10,35,बृहत्साम तथा साम्नां गायत्री छन्दसामहम्।\n\nमा...,"ఇంతకు పూర్వం శ్రీ కృష్ణుడు వేదములలో, అద్భుతమైన..."
652,18,37,यत्तदग्रे विषमिव परिणामेऽमृतोपमम्।तत्सुखं सात्...,భారతదేశ ఉసిరికాయ (ఆమ్లా) ఆరోగ్యానికి లాభకారియై...
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
646,18,31,यया धर्ममधर्मं च कार्यं चाकार्यमेव च।अयथावत्प्...,వ్యక్తిగత మమకారాసక్తుల వలన రాజసిక బుద్ధి మిశ్ర...
570,16,7,प्रवृत्तिं च निवृत्तिं च जना न विदुरासुराः।न श...,ధర్మము అంటే ఒక వ్యక్తి యొక్క పరిశుద్ధతకు మరియు...
403,10,39,यच्चापि सर्वभूतानां बीजं तदहमर्जुन।\n\nन तदस्त...,సమస్త సృష్టికి నిమిత్త కారణము (efficient cause...
595,17,8,आयुःसत्त्वबलारोग्यसुखप्रीतिविवर्धनाः।रस्याः स्...,"14వ అధ్యాయము, 6వ శ్లోకములో, శ్రీ కృష్ణుడు సత్త..."
589,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,"ఎవ్వరూ కూడా శ్రద్ధ/విశ్వాసము లేకుండా ఉండరు, ఎం..."


## Euclidean Distance

In [26]:
def mean_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [27]:
verse_index=mean_ed(query_embedding)

In [28]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
589,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,"ఎవ్వరూ కూడా శ్రద్ధ/విశ్వాసము లేకుండా ఉండరు, ఎం..."
570,16,7,प्रवृत्तिं च निवृत्तिं च जना न विदुरासुराः।न श...,ధర్మము అంటే ఒక వ్యక్తి యొక్క పరిశుద్ధతకు మరియు...
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,"లభతే అంటే ""వారు పొందెదరు"" అని. దేవతల యొక్క భ..."
262,6,37,अर्जुन उवाच\n\nअयतिः श्रद्धयोपेतो योगाच्चलितमा...,భగవత్ ప్రాప్తి దిశగా ప్రయాణం 'శ్రద్ధ' తోనే మొద...
459,11,53,नाहं वेदैर्न तपसा न दानेन न चेज्यया।\n\nशक्य ए...,"అర్జునుడికి విశ్వ రూపము చూపించిన పిదప, ఆ విశ్వ..."
458,11,52,श्री भगवानुवाच\n\nसुदुर्दर्शमिदं रूपं दृष्टवान...,"అర్జునుడికి విశ్వ రూపము చూపించిన పిదప, ఆ విశ్వ..."
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
656,18,41,ब्राह्मणक्षत्रियविशां शूद्राणां च परंतप।कर्माण...,సరిగ్గా సరిపోయే వృత్తి దొరకటం అంటే సరిగ్గా సరి...
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...


# Line Matching

## Cosine Similarity

In [29]:
def line_cosine(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[cosine_similarity(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [30]:
verse_index=line_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...
399,10,35,बृहत्साम तथा साम्नां गायत्री छन्दसामहम्।\n\nमा...,"ఇంతకు పూర్వం శ్రీ కృష్ణుడు వేదములలో, అద్భుతమైన..."
652,18,37,यत्तदग्रे विषमिव परिणामेऽमृतोपमम्।तत्सुखं सात्...,భారతదేశ ఉసిరికాయ (ఆమ్లా) ఆరోగ్యానికి లాభకారియై...
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
646,18,31,यया धर्ममधर्मं च कार्यं चाकार्यमेव च।अयथावत्प्...,వ్యక్తిగత మమకారాసక్తుల వలన రాజసిక బుద్ధి మిశ్ర...
570,16,7,प्रवृत्तिं च निवृत्तिं च जना न विदुरासुराः।न श...,ధర్మము అంటే ఒక వ్యక్తి యొక్క పరిశుద్ధతకు మరియు...
403,10,39,यच्चापि सर्वभूतानां बीजं तदहमर्जुन।\n\nन तदस्त...,సమస్త సృష్టికి నిమిత్త కారణము (efficient cause...
595,17,8,आयुःसत्त्वबलारोग्यसुखप्रीतिविवर्धनाः।रस्याः स्...,"14వ అధ్యాయము, 6వ శ్లోకములో, శ్రీ కృష్ణుడు సత్త..."
589,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,"ఎవ్వరూ కూడా శ్రద్ధ/విశ్వాసము లేకుండా ఉండరు, ఎం..."


## Euclidean Distance

In [31]:
def line_ed(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[ed(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [32]:
verse_index=line_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
589,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,"ఎవ్వరూ కూడా శ్రద్ధ/విశ్వాసము లేకుండా ఉండరు, ఎం..."
570,16,7,प्रवृत्तिं च निवृत्तिं च जना न विदुरासुराः।न श...,ధర్మము అంటే ఒక వ్యక్తి యొక్క పరిశుద్ధతకు మరియు...
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,"లభతే అంటే ""వారు పొందెదరు"" అని. దేవతల యొక్క భ..."
262,6,37,अर्जुन उवाच\n\nअयतिः श्रद्धयोपेतो योगाच्चलितमा...,భగవత్ ప్రాప్తి దిశగా ప్రయాణం 'శ్రద్ధ' తోనే మొద...
459,11,53,नाहं वेदैर्न तपसा न दानेन न चेज्यया।\n\nशक्य ए...,"అర్జునుడికి విశ్వ రూపము చూపించిన పిదప, ఆ విశ్వ..."
458,11,52,श्री भगवानुवाच\n\nसुदुर्दर्शमिदं रूपं दृष्टवान...,"అర్జునుడికి విశ్వ రూపము చూపించిన పిదప, ఆ విశ్వ..."
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
656,18,41,ब्राह्मणक्षत्रियविशां शूद्राणां च परंतप।कर्माण...,సరిగ్గా సరిపోయే వృత్తి దొరకటం అంటే సరిగ్గా సరి...
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...


# Whole Part

## Cosine Similarity

In [33]:
def whole_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [34]:
verse_index=whole_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...
399,10,35,बृहत्साम तथा साम्नां गायत्री छन्दसामहम्।\n\nमा...,"ఇంతకు పూర్వం శ్రీ కృష్ణుడు వేదములలో, అద్భుతమైన..."
652,18,37,यत्तदग्रे विषमिव परिणामेऽमृतोपमम्।तत्सुखं सात्...,భారతదేశ ఉసిరికాయ (ఆమ్లా) ఆరోగ్యానికి లాభకారియై...
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
646,18,31,यया धर्ममधर्मं च कार्यं चाकार्यमेव च।अयथावत्प्...,వ్యక్తిగత మమకారాసక్తుల వలన రాజసిక బుద్ధి మిశ్ర...
570,16,7,प्रवृत्तिं च निवृत्तिं च जना न विदुरासुराः।न श...,ధర్మము అంటే ఒక వ్యక్తి యొక్క పరిశుద్ధతకు మరియు...
403,10,39,यच्चापि सर्वभूतानां बीजं तदहमर्जुन।\n\nन तदस्त...,సమస్త సృష్టికి నిమిత్త కారణము (efficient cause...
595,17,8,आयुःसत्त्वबलारोग्यसुखप्रीतिविवर्धनाः।रस्याः स्...,"14వ అధ్యాయము, 6వ శ్లోకములో, శ్రీ కృష్ణుడు సత్త..."
589,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,"ఎవ్వరూ కూడా శ్రద్ధ/విశ్వాసము లేకుండా ఉండరు, ఎం..."


## Euclidean Distance

In [35]:
def whole_ed(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [36]:
verse_index=whole_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
8,1,11,अयनेषु च सर्वेषु यथाभागमवस्थिताः।\n\nभीष्ममेवा...,"శత్రువులకు అసాధ్యుడైన భీష్ముడిని, తన సైన్యానిక..."
428,11,22,रुद्रादित्या वसवो ये च साध्या\n\nविश्वेऽश्िवनौ...,వీరందరూ వ్యక్తులు తమతమ స్థానములను భగవంతుడి అను...
482,13,1,अर्जुन उवाच\n\nप्रकृतिं पुरुषं चैव क्षेत्रं क्...,"\n\n \n(*1) కొన్ని భగవద్గీత కోశములలో, ఈ శ్లో..."
446,11,40,नमः पुरस्तादथ पृष्ठतस्ते\n\nनमोऽस्तु ते सर्वत ...,"శ్రీ కృష్ణుడిని ""అనంత-వీర్య"" (అనంతమైన సామర్ద్య..."
419,11,13,तत्रैकस्थं जगत्कृत्स्नं प्रविभक्तमनेकधा।\n\nअप...,ఆ విశ్వ రూపములోని మహాద్భుతమైన ఆశ్చర్యములను వివ...
12,1,15,पाञ्चजन्यं हृषीकेशो देवदत्तं धनंजयः।\n\nपौण्ड्...,"శ్రీ కృష్ణుడికి వాడబడిన 'హృషీకేశః' అన్న పదం, ఆ..."
323,8,21,अव्यक्तोऽक्षर इत्युक्तस्तमाहुः परमां गतिम्।\n\...,ఆధ్యాత్మిక లోకం లోని దివ్య ఆకాశాన్నే పరవ్యోమము...
424,11,18,त्वमक्षरं परमं वेदितव्यं\n\nत्वमस्य विश्वस्य प...,పరమేశ్వరునిగా శ్రీ కృష్ణుడి సార్వభౌమాధికారాన్న...
134,3,23,यदि ह्यहं न वर्तेयं जातु कर्मण्यतन्द्रितः।\n\n...,"ఈ భూలోకంలో తన దివ్య లీలలలో భాగంగా, శ్రీ కృష్ణు..."
409,11,3,एवमेतद्यथात्थ त्वमात्मानं परमेश्वर।\n\nद्रष्टु...,అర్జునుడు శ్రీ కృష్ణుడి దివ్య వ్యక్తిత్వపు యదా...


# Clustering

## Max Pooling

In [37]:
kmeans_max = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_max)

In [38]:
def kmeans_max_cluster(query_embedding_list):
    label=kmeans_max.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_max.labels_[i]==label]
    return verse_index

In [39]:
verse_index=kmeans_max_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,"లభతే అంటే ""వారు పొందెదరు"" అని. దేవతల యొక్క భ..."


## Mean Pooling

In [40]:
kmeans_mean = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_mean)

In [41]:
def kmeans_mean_cluster(query_embedding_list):
    label=kmeans_mean.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_mean.labels_[i]==label]
    return verse_index

In [42]:
verse_index=kmeans_mean_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,"లభతే అంటే ""వారు పొందెదరు"" అని. దేవతల యొక్క భ..."


## Whole

In [43]:
kmeans_whole = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_whole)

In [44]:
def kmeans_whole_cluster(query_embedding_list):
    label=kmeans_whole.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_whole.labels_[i]==label]
    return verse_index

In [45]:
verse_index=kmeans_whole_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,"లభతే అంటే ""వారు పొందెదరు"" అని. దేవతల యొక్క భ..."
